## Imports

In [10]:
import json
from collections import Counter
import random
import openai  # Only if you're doing recipe suggestions
import os

## Universal Variables

In [ ]:
#setup openai key
openai_api_key = os.environ['OPENAI_API_KEY']

## Define Functions

In [6]:
# Load past grocery lists
def load_grocery_lists(filepath):
    with open(filepath, 'r') as f:
        grocery_lists = json.load(f)
    return grocery_lists

# Normalize & Flatten Lists
def normalize_items(grocery_lists):
    flat_list = []
    for sublist in grocery_lists:
        for item in sublist:
            clean_item = item.lower().strip()
            flat_list.append(clean_item)
    return flat_list

# Generate Next Week's Grocery List
def generate_next_week_list(flat_list, top_n=20):
    freq = Counter(flat_list)
    most_common = freq.most_common(top_n)
    return [item for item, count in most_common]

# Generate Recipe Ideas
def suggest_recipe(ingredients, openai_api_key=None):
    if not openai_api_key:
        return "No API key provided, skipping recipe generation."

    openai.api_key = openai_api_key
    prompt = f"Give me a healthy recipe using these ingredients: {', '.join(ingredients)}"

    try:
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}]
        )
        return response['choices'][0]['message']['content']
    except Exception as e:
        return f"Error generating recipe: {e}"


## Define Main Function

In [12]:
# Main Runner
if __name__ == "__main__":
    # Replace with your own file path
    filepath = "grocery_lists.json"
    grocery_lists = load_grocery_lists('C:/Users/MehakGanju/Documents/Python Scripts/SmartHeath POC/grocery_lists.json')

    flat_items = normalize_items(grocery_lists)
    next_week_list = generate_next_week_list(flat_items, top_n=20)

    print("✅ Suggested Grocery List for Next Week:")
    for item in next_week_list:
        print("-", item)

    # Optional: Get a random group of 4 items to suggest a recipe
    recipe_ingredients = random.sample(next_week_list, 4)
    recipe = suggest_recipe(recipe_ingredients, openai_api_key)
    print("\n🍳 Recipe Suggestion Based on Ingredients:")
    print(recipe)

✅ Suggested Grocery List for Next Week:
- rice
- chia seeds
- spinach
- cream cheese
- bagels
- ground turkey
- banana peppers
- carrots
- zuchini
- milk
- bananas
- bell peppers
- pepper jack cheese
- greek yogurt
- grapes
- ranch
- chicken nuggets
- turkey burgers
- oats
- blackberries

🍳 Recipe Suggestion Based on Ingredients:
Recipe: Spicy Carrot and Oats Rice

Ingredients:
- 1 cup of oats
- 2 medium-sized carrots, peeled and diced
- 1 cup of rice
- 4 banana peppers, sliced
- Salt to taste
- 2 tablespoons of olive oil
- 1 tablespoon of minced garlic
- 1 medium-sized onion, finely chopped
- 1 tablespoon of cumin seeds

Directions:
1. Rinse the rice under cold water until the water runs clear. This helps to remove the excess starch.
2. In a saucepan, add 2 cups of water and bring to a boil. Add the rice and a pinch of salt, and cook over medium heat for about 15-20 minutes, until the rice is cooked. Set aside.
3. In another saucepan, bring 2 cups of water to a boil. Add the oats and 

## Chat-GPT Implementation

In [23]:
def load_grocery_lists(filepath="grocery_lists.json"):
    with open(filepath, 'r') as f:
        return json.load(f)


def generate_smart_grocery_list_and_recipes(past_grocery_lists):
    all_items = [item for week in past_grocery_lists for item in week]
    input_str = ", ".join(sorted(set(all_items)))

    goal_instruction = f"Please tailor everything to a '{user_goal}' diet." if user_goal else ""

    prompt = f"""
You are an AI nutritionist and meal planner. Based on this user's previous 30 grocery lists:

{input_str}

1. Suggest 3-5 meal recipes they might be making regularly.
2. Create a smart grocery list for the upcoming week that includes all the ingredients needed for those recipes, avoids duplicates from long-lasting pantry items (like oats, peanut butter), and includes reasonable variety.
3. Make sure the grocery list order makes sense so group things like veggies, fruits or breads like they would be in a grocery store next to each other.
4. Keep the grocery list between 20 and 30 items.
5. Output the results in this format:

Recipes:
- Recipe 1: [short description]
- Recipe 2: ...

Grocery List:
- item 1
- item 2
...
    """

    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a helpful AI nutritionist."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content


In [24]:
if __name__ == "__main__":
    past_lists = load_grocery_lists()
    result = generate_smart_grocery_list_and_recipes(past_lists)
    print(result)

Recipes:
- Recipe 1: Chicken Fajitas - A delicious and healthy Mexican dish using chicken breast, bell peppers, onions and tortillas.
- Recipe 2: Protein Oatmeal Bowl - A protein-rich breakfast that incorporates oats, Greek yogurt, chia seeds and fresh fruits like apple and banana.
- Recipe 3: Salmon Salad - A nutritious salad mix with salmon, avocado, bell peppers and your choice of dressing.
- Recipe 4: Turkey Burger - A healthier option compared to regular burgers, the turkey burger can be enjoyed with a serving of veggies.
- Recipe 5: Peanut Butter and Cream Cheese Bagel - A quick and easy breakfast or snack that combines the creaminess of cream cheese with the rich taste of peanut butter.

Grocery List:
1. Chicken Breast
2. Red and Green Bell Peppers
3. White Onion
4. Small Tortillas
5. Greek Yogurt
6. Oats
7. Chia Seeds
8. Apples
9. Bananas
10. Salmon Fillet
11. Avocado
12. Salad Mix
13. Ground Turkey
14. Whole Wheat Buns
15. Bagels
16. Cream Cheese
17. Peanut Butter
18. Lettuce
